In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Загрузка даних
data = pd.read_csv('GlobalYouTubeStatistics.csv', delimiter=',', encoding='latin1')

# ВИдалення нульових значень
data = data.dropna()

# Перетворення категоріальних даних на float
label_encoder = LabelEncoder()
data['uploads'] = label_encoder.fit_transform(data['uploads'])
data['uploads'] = data['uploads'].astype(float)
data['Country'] = label_encoder.fit_transform(data['Country'])
data['Country'] = data['Country'].astype(float)
data['channel_type'] = label_encoder.fit_transform(data['channel_type'])
data['channel_type'] = data['channel_type'].astype(float)

# Розбиття даних
X = data[['uploads', 'Country']]
y = data['channel_type']

# Розділення на навчальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from joblib import Parallel, delayed

# Параметри для Decision Tree моделі
tree_params = {
    'criterion': 'gini',
    'max_depth': None,
    'min_samples_split': 2,
    'min_samples_leaf': 1
}

# Кількість моделей у гомогенному ансамблі
n_estimators = 3

# Функція для тренування одного Decision Tree
def train_tree(seed):
    model = DecisionTreeClassifier(random_state=seed, **tree_params)
    return model

# Створення гомогенного ансамблю паралельно
models = Parallel(n_jobs=-1)(delayed(train_tree)(seed) for seed in range(n_estimators))

# Створення паралельного гомогенного ансамблю на основі Decision Tree
parallel_tree_ensemble = BaggingClassifier(estimator=None, n_estimators=n_estimators, n_jobs=-1)

# Тренування гомогенного ансамблю
parallel_tree_ensemble.fit(X_train, y_train)

BaggingClassifier(n_estimators=3, n_jobs=-1)

In [12]:
from sklearn.ensemble import RandomForestClassifier

# Параметри для RandomForest моделі
rf_params = {
    'n_estimators': 3,
    'criterion': 'gini',
    'max_depth': None,
    'min_samples_split': 2,
    'min_samples_leaf': 1
}

# Кількість моделей у гомогенному ансамблі
n_estimators = 3

# Створення гомогенного ансамблю
sequential_rf_ensemble = []

# Функція для тренування одного RandomForest
def train_rf(seed):
    model = RandomForestClassifier(random_state=seed, **rf_params)
    return model

# Тренування та додавання кожен моделей RandomForest до гомогенного ансамблю
for seed in range(n_estimators):
    model = train_rf(seed)
    sequential_rf_ensemble.append(model)
    
# Тренування кожної моделі в ансамблі на тренувальних даних
for model in sequential_rf_ensemble:
    model.fit(X_train, y_train)

In [14]:
from sklearn.metrics import accuracy_score

# Прогноз для гомогенного ансамблю на основі Decision Tree
predictions_decision_tree = parallel_tree_ensemble.predict(X_test)

# Прогноз для гомогенного ансамблю на основі RandomForest
predictions_rf_ensemble = [model.predict(X_test) for model in sequential_rf_ensemble]

# Точність для гомогенного ансамблю на основі Decision Tree
accuracy_decision_tree = accuracy_score(y_test, predictions_decision_tree)
print(f"Accuracy for Parallel Decision Tree Ensemble: {accuracy_decision_tree}")

# Точність для гомогенного ансамблю на основі RandomForest
accuracy_rf_ensemble = accuracy_score(y_test, predictions_rf_ensemble[0])
print(f"Accuracy for Sequential RandomForest Ensemble: {accuracy_rf_ensemble}")

Accuracy for Parallel Decision Tree Ensemble: 0.34234234234234234
Accuracy for Sequential RandomForest Ensemble: 0.2972972972972973


In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

# Параметр для оптимізації ансамблю на основі Decision Tree
param_grid_decision_tree = {
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Нова модель Decision Tree
base_model_decision_tree = DecisionTreeClassifier()

# GridSearchCV для оптимізації гіперпараметрів Decision Tree
grid_search_decision_tree = GridSearchCV(base_model_decision_tree, param_grid_decision_tree, cv=2)
grid_search_decision_tree.fit(X_train, y_train)

# Вивід найкращих параметрів
best_params_decision_tree = grid_search_decision_tree.best_params_
print(f"Best parameters for Parallel Decision Tree Ensemble: {best_params_decision_tree}")

# Використання нової моделі в ансамблі
best_decision_tree_model = grid_search_decision_tree.best_estimator_
parallel_tree_ensemble_optimized = BaggingClassifier(estimator=best_decision_tree_model, n_estimators=10, n_jobs=-1)

# Тренування гомогенного ансамблю
parallel_tree_ensemble_optimized.fit(X_train, y_train)

# Точність
accuracy_parallel_tree_ensemble = parallel_tree_ensemble_optimized.score(X_test, y_test)
print(f"Accuracy for optimized Parallel Decision Tree Ensemble: {accuracy_parallel_tree_ensemble}")

Best parameters for Parallel Decision Tree Ensemble: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2}
Accuracy for optimized Parallel Decision Tree Ensemble: 0.36036036036036034


In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Параметри для оптимізації ансамблю на основі RandomForest
param_grid_rf = {
    'n_estimators': [10, 50, 100],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV для оптимізації RandomForest
grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=2)
grid_search_rf.fit(X_train, y_train)

# Вивід найкращих параметрів RandomForest
best_params_rf = grid_search_rf.best_params_
print(f"Best parameters for Sequential RandomForest Ensemble: {best_params_rf}")

# Використання оптимізованої моделі RandomForest в ансамблі
best_rf_model = grid_search_rf.best_estimator_
sequential_rf_ensemble_optimized = [best_rf_model] * n_estimators

# Тренування ансамблю
for model in sequential_rf_ensemble_optimized:
    model.fit(X_train, y_train)
    
# Прогноз
predictions_rf_ensemble = [model.predict(X_test) for model in sequential_rf_ensemble_optimized]

# Точність
accuracy_rf_ensemble = accuracy_score(y_test, predictions_rf_ensemble[0])
print(f"Accuracy for optimized Sequential RandomForest Ensemble: {accuracy_rf_ensemble}")

Best parameters for Sequential RandomForest Ensemble: {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 100}
Accuracy for optimized Sequential RandomForest Ensemble: 0.42342342342342343


In [30]:
import numpy as np

# Отримуємо важливість ознак для кожного Decision Tree в ансамблі
feature_importances_decision_trees = [tree.feature_importances_ for tree in parallel_tree_ensemble_optimized.estimators_]

# Середнє значення важливості ознак по всім Decision Tree
average_feature_importance_decision_trees = np.mean(feature_importances_decision_trees, axis=0)

# Створюємо словник, де ключі - назви ознак, значення - їх середнє значення важливості
feature_importance_dict_decision_trees = dict(zip(X_train.columns, average_feature_importance_decision_trees))

# Виводимо важливість ознак для ансамблю на основі Decision Tree
print("Feature Importance for Parallel Decision Tree Ensemble:")
for feature, importance in feature_importance_dict_decision_trees.items():
    print(f"{feature}: {importance}")

Feature Importance for Parallel Decision Tree Ensemble:
uploads: 0.7544387630095539
Country: 0.24556123699044602


In [32]:
# Отримуємо важливість ознак для RandomForest
feature_importance_rf = best_rf_model.feature_importances_

# Створюємо словник, де ключі - назви ознак, значення - їх важливість
feature_importance_dict_rf = dict(zip(X_train.columns, feature_importance_rf))

# Виводимо важливість ознак для RandomForest
print("Feature Importance for RandomForest:")
for feature, importance in feature_importance_dict_rf.items():
    print(f"{feature}: {importance}")

Feature Importance for RandomForest:
uploads: 0.733639771966949
Country: 0.2663602280330511
